In [169]:
# In this homework, we will use the lead scoring dataset Bank Marketing dataset.
import pandas as pd

df = pd.read_csv('course_lead_scoring.csv')
# In this dataset our desired target for classification task will be
# converted variable - has the client signed up to the platform or not.

In [170]:
df.describe()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
count,1462.000000,1281.000000,1462.000000,1462.000000,1462.000000
mean,2.031464,59886.273224,2.976744,0.506108,0.619015
std,1.449717,15070.140389,1.681564,0.288465,0.485795
min,0.000000,13929.000000,0.000000,0.000000,0.000000
25%,1.000000,49698.000000,2.000000,0.262500,0.000000
50%,2.000000,60148.000000,3.000000,0.510000,1.000000
75%,3.000000,69639.000000,4.000000,0.750000,1.000000
max,9.000000,109899.000000,11.000000,1.000000,1.000000


In [171]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [172]:
print( df.dtypes)

mask = (df.dtypes.index != 'converted')
categorical = df.dtypes[ (df.dtypes == 'object') & (mask)].index
numerical = df.dtypes[(df.dtypes != 'object') & (mask)].index

print(categorical)
print(numerical)

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object
Index(['lead_source', 'industry', 'employment_status', 'location'], dtype='object')
Index(['number_of_courses_viewed', 'annual_income', 'interaction_count',
       'lead_score'],
      dtype='object')


In [173]:
# Data preparation
# Check if the missing values are presented in the features.
cols = pd.DataFrame( list( zip ( df.columns, df.isnull().sum(), df.isna().sum(), df.dtypes)) )
cols

,0,1,2,3
0,lead_source,128,128,object
1,industry,134,134,object
2,number_of_courses_viewed,0,0,int64
3,annual_income,181,181,float64
4,employment_status,100,100,object
5,location,63,63,object
6,interaction_count,0,0,int64
7,lead_score,0,0,float64
8,converted,0,0,int64


In [174]:
# If there are missing values:
# For caterogiral features, replace them with 'NA'
df_fill = df.copy()
#del df_fill.converted

for c in categorical:
  df_fill[c] = df[c].fillna('NA')

# For numerical features, replace with with 0.0
for c in numerical:
  df_fill[c] = df[c].fillna(0)

In [175]:
# Data preparation
# Check if the missing values are presented in the features.
cols = pd.DataFrame( list( zip ( df_fill.columns, df_fill.isnull().sum(), df_fill.isna().sum(), df_fill.dtypes)) )
cols

,0,1,2,3
0,lead_source,0,0,object
1,industry,0,0,object
2,number_of_courses_viewed,0,0,int64
3,annual_income,0,0,float64
4,employment_status,0,0,object
5,location,0,0,object
6,interaction_count,0,0,int64
7,lead_score,0,0,float64
8,converted,0,0,int64


In [176]:
# Question 1: What is the most frequent observation (mode) for the column industry?
print( df_fill.industry.mode()) #??


print( df_fill.industry.value_counts())

print( df_fill.lead_score.mode()) #??
print( df_fill.lead_score.value_counts()) #??

0    retail
Name: industry, dtype: object
industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64
0    0.6
Name: lead_score, dtype: float64
lead_score
0.60    24
0.82    22
0.02    21
0.20    21
0.41    21
        ..
0.19     9
0.23     8
0.12     7
0.77     7
1.00     5
Name: count, Length: 101, dtype: int64


In [177]:
# Question 2 : Create the correlation matrix for the numerical features of your dataset.
# In a correlation matrix, you compute the correlation coefficient between every pair of features.
df_numerical = df_fill[numerical]

print(numerical)

print(df_fill[:1].T)

Index(['number_of_courses_viewed', 'annual_income', 'interaction_count',
       'lead_score'],
      dtype='object')
                                      0
lead_source                    paid_ads
industry                             NA
number_of_courses_viewed              1
annual_income                   79450.0
employment_status            unemployed
location                  south_america
interaction_count                     4
lead_score                         0.94
converted                             1


In [178]:
corr_matrix = df_fill[numerical].corr()
corr_matrix
# What are the two features that have the biggest correlation? : annual_income and interaction_count

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [179]:
# Split the data
# Split your data in train/val/test sets with 60%/20%/20% distribution.
# Use Scikit-Learn for that (the train_test_split function) and
# set the seed to 42.
# Make sure that the target value y is not in your dataframe.

from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df_fill, test_size=0.2, random_state=42)

print(df_full_train.shape)
print(df_test.shape)

# 0.25?  print(len(df_full_train))
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

print(df_train.shape)
print(df_val.shape)

print(len(df_train) + len(df_val) + len(df_test) , len(df))

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

(1169, 9)
(293, 9)
(876, 9)
(293, 9)
1462 1462


In [180]:
df_train.columns
df_val.columns
df_test.columns


Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score'],
      dtype='object')

In [181]:
# Question 3 : Calculate the mutual information score between
# y and other categorical variables in the dataset.
# Use the training set only.
# Round the scores to 2 decimals using round(score, 2).
# Which of these variables has the biggest mutual information score?
# R => lead_source

from sklearn.metrics import mutual_info_score

# print(df_train[categorical].isnull().sum())

mi = {}
for c in categorical:
  mi[c] = mutual_info_score(df_train[c], y_train)
  print(c, round(mi[c], 2))

#mi = mutual_info_score(df_train[categorical], y_train)

lead_source 0.04
industry 0.01
employment_status 0.01
location 0.0


In [182]:
# R => lead_source
def mutual_info_converted(series):
  return mutual_info_score(y_train, series)

mi2 = df_train[categorical].apply(mutual_info_converted)

mi2.round(2).sort_values(ascending=False)

,0
lead_source,0.04
industry,0.01
employment_status,0.01
location,0.00


In [183]:
# Question 4 : Now let's train a logistic regression.
# Remember that we have several categorical variables in the dataset.
# Include them using one-hot encoding.

from sklearn.feature_extraction import DictVectorizer

df_train_dict = df_train.to_dict(orient='records')
df_train_dict[0]
categorical_data_train = df_train_dict[:100]

dv = DictVectorizer(sparse=False)

dv.fit(categorical_data_train)

X_train = dv.transform(df_train_dict)
print( X_train[:1], X_train.shape)

X_val = dv.transform(df_val.to_dict(orient='records'))
print( X_train[:1], X_train.shape)


[[5.8472e+04 0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  1.0000e+00 0.0000e+00 5.0000e+00 3.0000e-02 0.0000e+00 0.0000e+00
  0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00]] (876, 31)
[[5.8472e+04 0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  1.0000e+00 0.0000e+00 5.0000e+00 3.0000e-02 0.0000e+00 0.0000e+00
  0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00]] (876, 31)


In [184]:
# Fit the model on the training dataset.
# To make sure the results are reproducible across different versions of
# Scikit-Learn, fit the model with these parameters:
# model = LogisticRegression(
    #solver='liblinear', C=1.0, max_iter=1000, random_state=42)

from sklearn.linear_model import LogisticRegression

print(X_train.shape, y_train.shape)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

model.fit(X_train, y_train)

w0 = model.intercept_
W = model.coef_

print(w0, W)

(876, 31) (876,)
[-0.06914728] [[-1.77843867e-05 -1.47154423e-02  3.39095225e-02  2.66248432e-03
   1.15238518e-02 -1.02527697e-01 -2.48510995e-02  4.93604222e-02
  -2.01258344e-02 -1.34214865e-02 -3.00232200e-03 -9.25991830e-03
  -3.17957304e-02 -1.60513114e-02  3.11339155e-01  5.12012528e-02
   2.01511698e-02 -1.20346284e-02 -1.16021521e-02 -1.15251880e-01
   7.95303436e-02 -2.99401329e-02  3.95843295e-03 -1.14296944e-02
  -1.12457415e-02 -5.59987025e-03  8.26402635e-03  5.58598769e-03
  -3.33967159e-02 -2.52837052e-02  4.53752887e-01]]


In [185]:
# Hard Predictions
model.predict(X_train)

# Soft Predictions
model.predict_proba(X_train)

array([[0.42085657, 0.57914343],
       [0.12716509, 0.87283491],
       [0.41183893, 0.58816107],
       ...,
       [0.25265784, 0.74734216],
       [0.3302157 , 0.6697843 ],
       [0.14407823, 0.85592177]])

In [252]:
# Calculate the accuracy on the validation dataset and
# round it to 2 decimal digits.

y_predict = model.predict_proba(X_val)

converted_decision = (y_predict[:, 1] > 0.5 )

df_val[converted_decision]

original_accuracy = (y_val == converted_decision).mean().round(3)
original_accuracy


np.float64(0.7)

In [187]:
X_test = dv.transform(df_test.to_dict(orient='records'))
y_predict_test = model.predict_proba(X_test)

converted_decision_test = (y_predict_test[:, 1] > 0.5 )

(y_test == converted_decision_test).mean().round(2)

np.float64(0.73)

In [194]:
# Question 5
# Let's find the least useful feature using the feature elimination technique.
# Train a model using the same features and parameters as in Q4 (without rounding).
# Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
# For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
# Which of following feature has the smallest difference?
accuracy_score=[]

# del df_x['converted']
# del df_v['converted']
print(df_x.columns, 'VAL \n', df_v.columns)


['lead_source', 'industry', 'employment_status', 'location', 'number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score'],
      dtype='object') VAL 
 Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score'],
      dtype='object')


In [222]:
# Calculate the accuracy on the validation dataset and
# round it to 2 decimal digits.

def get_accuracy(model, df_val):
  y_predict = model.predict_proba(df_val)

  converted_decision = (y_predict[:, 1] > 0.5 )

  accuracy = (y_val == converted_decision).mean().round(2)
  #print(accuracy)
  return  accuracy


In [236]:
def elimination(features, df_x, df_v):
  dv_s = DictVectorizer(sparse=False)
  model_small = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

  for f in features:
    df_x = df_x.drop(columns=[f])
    df_v = df_v.drop(columns=[f])
    #print(df_x.columns)

    if len(df_x.columns) == 0:
      #print('break')
      break


    x_dict = df_x.to_dict(orient='records')
    dv_s.fit(x_dict)

    x_cat = dv_s.transform(x_dict)
    val_cat = dv_s.transform(df_v.to_dict(orient='records'))

    model_small.fit(x_cat, y_train)
    w0 = model_small.intercept_
    W = model_small.coef_
    #print(w0, W)

    accurr = get_accuracy(model_small, val_cat)
    print(f, ' : ', accurr, (original_accuracy-accurr).round(2) )
    #return w0, W

features = list(categorical) + list(numerical)
print(features)

df_x = df_train.copy()
df_v = df_val.copy()
elimination(features, df_x, df_v)


['lead_source', 'industry', 'employment_status', 'location', 'number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
lead_source  :  0.7 0.0
industry  :  0.7 0.0
employment_status  :  0.69 0.01
location  :  0.69 0.01
number_of_courses_viewed  :  0.56 0.14
annual_income  :  0.68 0.02
interaction_count  :  0.63 0.07


In [235]:
def score_one(cols, df_x, df_v):
  dv_s = DictVectorizer(sparse=False)
  model_small = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

  df_x = df_x[cols]
  df_v = df_v[cols]
    #print(df_x.columns)

  x_dict = df_x.to_dict(orient='records')
  dv_s.fit(x_dict)

  x_cat = dv_s.transform(x_dict)
  val_cat = dv_s.transform(df_v.to_dict(orient='records'))

  model_small.fit(x_cat, y_train)
  w0 = model_small.intercept_
  W = model_small.coef_

  accurr = get_accuracy(model_small, val_cat)
  print(f, ' : ', accurr, (original_accuracy-accurr).round(2) )
    #return w0, W

df_x = df_train.copy()
df_v = df_val.copy()
score_one(['lead_score'], df_x, df_v)

lead_score  :  0.63 0.07


In [251]:
# Question 6
# Now let's train a regularized logistic regression.
# Let's try the following values of the parameter
# C: [0.01, 0.1, 1, 10, 100].
# Train models using all the features as in Q4.
# Calculate the accuracy on the validation dataset and
# round it to 3 decimal digits.
# Which of these C leads to the best accuracy on the validation set?

def train_model(X_train, y_train, df_val, c):
  model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)

  model.fit(X_train, y_train)

  w0 = model.intercept_
  W = model.coef_
  #print(w0, W)
  score = get_accuracy(model, df_val)
  return score

for c in [0.01, 0.1, 1, 10, 100]:
  score = train_model(X_train, y_train, X_val, c)
  print(c, ' : ', score)


0.01  :  0.7
0.1  :  0.7
1  :  0.7
10  :  0.7
100  :  0.7


1. Is normal to have same accuracy on dif c?
2. I dont explore y forme
3. Why linear methods have "a good beheivour"?